<a href="https://colab.research.google.com/github/yjunej/Boostcamp-AI-Tech/blob/main/P-Stage/P2/P2_%ED%95%9C%EA%B5%AD%EC%96%B4_%ED%86%A0%ED%81%AC%EB%82%98%EC%9D%B4%EC%A7%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국어 Tokenizing

> 작성자      
```
* 김성현 (bananaband657@gmail.com)  
김바다 (qkek983@gmail.com)
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
```
[CC BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/2.0/kr/)


한국어에서의 다양한 tokenizing 방식을 실습해보겠습니다.   

한국어는 다음의 단계로 tokenizing이 가능합니다.

1. 어절 단위
2. 형태소 단위
3. 음절 단위
4. 자소 단위
5. WordPiece 단위

## 0. 실습용 데이터 준비

실습을 위해 한국어 wikipedia 파일을 가져오도록 하겠습니다.   
본 wikipedia 파일은 앞선 전처리 실습을 통해 전처리가 완료된 파일입니다.   


In [1]:
!mkdir my_data

In [2]:
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" -o my_data/wiki_20190620_small.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    498      0 --:--:-- --:--:-- --:--:--   498
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1323k  100 1323k    0     0  1284k      0  0:00:01  0:00:01 --:--:-- 1284k


데이터를 확인해보겠습니다.

In [3]:
data = open('my_data/wiki_20190620_small.txt', 'r', encoding='utf-8')
# 'r' 은 read를 의미합니다.
# 본 파일은 encoding format을 UTF-8로 저장했기 때문에, UTF-8로 읽겠습니다.
# 한국어는 특히 encoding format이 맞지 않으면, 글자가 깨지는 현상이 나타납니다.

In [4]:
lines = data.readlines() # 전체 문장을 list에 저장하는 함수입니다.

In [5]:
for line in lines[0:10]:
    print(line)

제임스 얼 "지미" 카터 주니어는 민주당 출신 미국 39번째 대통령 이다.

지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.

조지아 공과대학교를 졸업하였다.

그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다.

1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다.

그의 별명이 "땅콩 농부" 로 알려졌다.

1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다.

대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다.

조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.

1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다.



## 1. 어절 단위 tokenizing

어절 단위 tokenizing은 모든 문장을 띄어쓰기 단위로 분리하는 것을 의미합니다.

"이순신은 조선 중기의 무신이다." -> ["이순신은", "조선", "중기의", "무신이다."]

In [6]:
text = "이순신은 조선 중기의 무신이다."
tokenized_text = text.split(" ")    # split 함수는 입력 string에 대해서 특정 string을 기반으로 분리해줍니다.
print(tokenized_text)  

['이순신은', '조선', '중기의', '무신이다.']


Tokenizing의 목적은 크게 두 가지입니다.  
1. 의미를 지닌 단위로 자연어를 분절
2. Model의 학습 시, 동일한 size로 입력

따라서, tokenizer는 특정 사이즈로 token의 개수를 조절하는 함수가 필수로 포함되어야 합니다.

이를 위해, token의 개수가 부족할 때는 padding 처리를 해주고,    
개수가 많을 때는 token을 잘라서 반환하는 함수를 구현하겠습니다.   

In [7]:
max_seq_length = 10
# padding
tokenized_text += ["padding"] * (max_seq_length - len(tokenized_text))
print(tokenized_text)

['이순신은', '조선', '중기의', '무신이다.', 'padding', 'padding', 'padding', 'padding', 'padding', 'padding']


In [8]:
max_seq_length = 2
# filtering
tokenized_text = tokenized_text[0:max_seq_length]
print(tokenized_text)

['이순신은', '조선']


위 코드를 이용해 tokenizer class를 만들어보겠습니다.

In [9]:
class Tokenizer:
    def __init__(self):
        self.tokenizer_type_list = ["word"]
        self.pad_token = "<pad>"
        self.max_seq_length = 10
        self.padding = False
    def tokenize(self, text, tokenizer_type): 
        assert tokenizer_type in self.tokenizer_type_list, "정의되지 않은 tokenizer_type입니다."
        if tokenizer_type == "word":
            tokenized_text = text.split(" ")
        if self.padding:
            tokenized_text += [self.pad_token] * (self.max_seq_length - len(tokenized_text))
            return tokenized_text[:self.max_seq_length]
        else:
            return tokenized_text[:self.max_seq_length]
    def batch_tokenize(self, texts, tokenizer_type):
        for i, text in enumerate(texts):
            texts[i] = self.tokenize(text, tokenizer_type)
        return texts

In [10]:
my_tokenizer = Tokenizer()
my_tokenizer.pad_token = "[PAD]"
my_tokenizer.max_seq_length = 10
my_tokenizer.padding = True

In [11]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "word"))
print(my_tokenizer.batch_tokenize(["이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다."], "word"))

['이순신은', '조선', '중기의', '무신이다.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[['이순신은', '조선', '중기의', '무신이다.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'], ['그는', '임진왜란을', '승리로', '이끌었다.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']]


## 형태소 단위 tokenizing

형태소 분석기로는 mecab을 사용하겠습니다.

In [12]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 460kB 29.6MB/s 
     |████████████████████████████████| 92kB 6.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [13]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Installing automake (A dependency for mecab-ko)
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Package

In [14]:
from konlpy.tag import Mecab

mecab = Mecab()
print(mecab.pos("아버지가방에들어가신다."))

[('아버지', 'NNG'), ('가', 'JKS'), ('방', 'NNG'), ('에', 'JKB'), ('들어가', 'VV'), ('신다', 'EP+EF'), ('.', 'SF')]


In [15]:
text = "이순신은 조선 중기의 무신이다."
# 이순신 -> PS
# 조선 -> DT TI
# 중기 -> TI
# 무신 -> OC
# 이순신 - 직업 - 무신
# 이순신 - 출생지 - 조선

tokenized_text = [lemma[0] for lemma in mecab.pos(text)]
print(tokenized_text)

['이순신', '은', '조선', '중기', '의', '무신', '이', '다', '.']


형태소 tokenizer도 class에 추가하겠습니다.

In [16]:
class Tokenizer:
    def __init__(self):
        self.tokenizer_type_list = ["word", "morph"]
        self.pad_token = "<pad>"
        self.max_seq_length = 10
        self.padding = False
    def tokenize(self, text, tokenizer_type): 
        assert tokenizer_type in self.tokenizer_type_list, "정의되지 않은 tokenizer_type입니다."
        if tokenizer_type == "word":
            tokenized_text = text.split(" ")
        elif tokenizer_type == "morph":
            tokenized_text = [lemma[0] for lemma in mecab.pos(text)]
        if self.padding:
            tokenized_text += [self.pad_token] * (self.max_seq_length - len(tokenized_text))
            return tokenized_text[:self.max_seq_length]
        else:
            return tokenized_text[:self.max_seq_length]
    def batch_tokenize(self, texts, tokenizer_type):
        for i, text in enumerate(texts):
            texts[i] = self.tokenize(text, tokenizer_type)
        return texts

In [17]:
my_tokenizer = Tokenizer()
my_tokenizer.pad_token = "[PAD]"
my_tokenizer.max_seq_length = 10
my_tokenizer.padding = True

In [18]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "morph"))
print(my_tokenizer.batch_tokenize(["이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다."], "morph"))

['이순신', '은', '조선', '중기', '의', '무신', '이', '다', '.', '[PAD]']
[['이순신', '은', '조선', '중기', '의', '무신', '이', '다', '.', '[PAD]'], ['그', '는', '임진왜란', '을', '승리', '로', '이끌', '었', '다', '.']]


## 음절 단위 tokenizing

음절 단위 tokenizing은 한 자연어를 한 글자씩 분리합니다.

In [19]:
text = "이순신은 조선 중기의 무신이다."
tokenized_text = list(text)    # split 함수는 입력 string에 대해서 특정 string을 기반으로 분리해줍니다.
print(tokenized_text)  

['이', '순', '신', '은', ' ', '조', '선', ' ', '중', '기', '의', ' ', '무', '신', '이', '다', '.']


In [21]:
class Tokenizer:
    def __init__(self):
        self.tokenizer_type_list = ["word", "morph", "syllable"]
        self.pad_token = "<pad>"
        self.max_seq_length = 10
        self.padding = False
    def tokenize(self, text, tokenizer_type): 
        assert tokenizer_type in self.tokenizer_type_list, "정의되지 않은 tokenizer_type입니다."
        if tokenizer_type == "word":
            tokenized_text = text.split(" ")
        elif tokenizer_type == "morph":
            tokenized_text = [lemma[0] for lemma in mecab.pos(text)]
        elif tokenizer_type == "syllable":
            tokenized_text = list(text)
        if self.padding:
            tokenized_text += [self.pad_token] * (self.max_seq_length - len(tokenized_text))
            return tokenized_text[:self.max_seq_length]
        else:
            return tokenized_text[:self.max_seq_length]
    def batch_tokenize(self, texts, tokenizer_type):
        for i, text in enumerate(texts):
            texts[i] = self.tokenize(text, tokenizer_type)
        return texts

In [22]:
my_tokenizer = Tokenizer()
my_tokenizer.pad_token = "[PAD]"
my_tokenizer.max_seq_length = 20
my_tokenizer.padding = True

In [23]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "syllable"))
print(my_tokenizer.batch_tokenize(["이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다."], "syllable"))

['이', '순', '신', '은', ' ', '조', '선', ' ', '중', '기', '의', ' ', '무', '신', '이', '다', '.', '[PAD]', '[PAD]', '[PAD]']
[['이', '순', '신', '은', ' ', '조', '선', ' ', '중', '기', '의', ' ', '무', '신', '이', '다', '.', '[PAD]', '[PAD]', '[PAD]'], ['그', '는', ' ', '임', '진', '왜', '란', '을', ' ', '승', '리', '로', ' ', '이', '끌', '었', '다', '.', '[PAD]', '[PAD]']]


## 자소 단위 tokenizing

한글은 하나의 문자도 최대 초성, 중성, 종성, 총 3개의 자소로 분리가 가능합니다.   
실습에서는 자소 분리를 위해 hgtk 라이브러리를 사용하겠습니다.

In [24]:
!pip install hgtk

  Created wheel for hgtk: filename=hgtk-0.1.3-py2.py3-none-any.whl size=6689 sha256=24bf9a333bded2f193a3fc8a15f8d94cbbb895441a30e15ee5e1828078660c97
  Stored in directory: /root/.cache/pip/wheels/73/72/06/6065a57fe68264f35d7e52e37f56831eb3e9ec75656880de20
Successfully built hgtk


In [25]:
import hgtk

In [26]:
text = "이순신은 조선 중기의 무신이다."
tokenized_text = list(hgtk.text.decompose(text))
print(tokenized_text)
# ㅇ ㅣ ㅅ ㅜ ㄴ ㅅ ㅣ ... 

['ㅇ', 'ㅣ', 'ᴥ', 'ㅅ', 'ㅜ', 'ㄴ', 'ᴥ', 'ㅅ', 'ㅣ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅡ', 'ㄴ', 'ᴥ', ' ', 'ㅈ', 'ㅗ', 'ᴥ', 'ㅅ', 'ㅓ', 'ㄴ', 'ᴥ', ' ', 'ㅈ', 'ㅜ', 'ㅇ', 'ᴥ', 'ㄱ', 'ㅣ', 'ᴥ', 'ㅇ', 'ㅢ', 'ᴥ', ' ', 'ㅁ', 'ㅜ', 'ᴥ', 'ㅅ', 'ㅣ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅣ', 'ᴥ', 'ㄷ', 'ㅏ', 'ᴥ', '.']


In [27]:
class Tokenizer:
    def __init__(self):
        self.tokenizer_type_list = ["word", "morph", "syllable", "jaso"]
        self.pad_token = "<pad>"
        self.max_seq_length = 10
        self.padding = False
    def tokenize(self, text, tokenizer_type): 
        assert tokenizer_type in self.tokenizer_type_list, "정의되지 않은 tokenizer_type입니다."
        if tokenizer_type == "word":
            tokenized_text = text.split(" ")
        elif tokenizer_type == "morph":
            tokenized_text = [lemma[0] for lemma in mecab.pos(text)]
        elif tokenizer_type == "syllable":
            tokenized_text = list(text)
        elif tokenizer_type == "jaso":
            tokenized_text = list(hgtk.text.decompose(text))
        if self.padding:
            tokenized_text += [self.pad_token] * (self.max_seq_length - len(tokenized_text))
            return tokenized_text[:self.max_seq_length]
        else:
            return tokenized_text[:self.max_seq_length]
    def batch_tokenize(self, texts, tokenizer_type):
        for i, text in enumerate(texts):
            texts[i] = self.tokenize(text, tokenizer_type)
        return texts

In [28]:
my_tokenizer = Tokenizer()
my_tokenizer.pad_token = "[PAD]"
my_tokenizer.max_seq_length = 20
my_tokenizer.padding = True

In [29]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "jaso"))
print(my_tokenizer.batch_tokenize(["이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다."], "jaso"))

['ㅇ', 'ㅣ', 'ᴥ', 'ㅅ', 'ㅜ', 'ㄴ', 'ᴥ', 'ㅅ', 'ㅣ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅡ', 'ㄴ', 'ᴥ', ' ', 'ㅈ', 'ㅗ', 'ᴥ', 'ㅅ']
[['ㅇ', 'ㅣ', 'ᴥ', 'ㅅ', 'ㅜ', 'ㄴ', 'ᴥ', 'ㅅ', 'ㅣ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅡ', 'ㄴ', 'ᴥ', ' ', 'ㅈ', 'ㅗ', 'ᴥ', 'ㅅ'], ['ㄱ', 'ㅡ', 'ᴥ', 'ㄴ', 'ㅡ', 'ㄴ', 'ᴥ', ' ', 'ㅇ', 'ㅣ', 'ㅁ', 'ᴥ', 'ㅈ', 'ㅣ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅙ', 'ᴥ', 'ㄹ']]


## WordPiece tokenizing

In [30]:
!pip install transformers

     |████████████████████████████████| 2.1MB 4.3MB/s 
     |████████████████████████████████| 870kB 18.4MB/s 
     |████████████████████████████████| 3.3MB 26.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=7509158f5f793a6c759f8ea6fc01b28558b0d55c519b4d3e696c5982b778a5ed
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [31]:
!mkdir wordPieceTokenizer

In [32]:

from tokenizers import BertWordPieceTokenizer

# Initialize an empty tokenizer
wp_tokenizer = BertWordPieceTokenizer(
    clean_text=True,    # [이순신, ##은, ' ', 조선]
    handle_chinese_chars=True,
    strip_accents=False,    # True: [YepHamza] -> [Yep, Hamza]
    lowercase=False,
)

# And then train
wp_tokenizer.train(
    files="my_data/wiki_20190620_small.txt",
    vocab_size=10000,
    min_frequency=2,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    limit_alphabet=1000,
    wordpieces_prefix="##"
)

# Save the files
wp_tokenizer.save_model("wordPieceTokenizer", "my_tokenizer")

['wordPieceTokenizer/my_tokenizer-vocab.txt']

In [33]:
print(wp_tokenizer.get_vocab_size())

10000


In [34]:
text = "이순신은 조선 중기의 무신이다."
tokenized_text = wp_tokenizer.encode(text)
print(tokenized_text)
print(tokenized_text.tokens)
print(tokenized_text.ids)

Encoding(num_tokens=10, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['이', '##순', '##신은', '조선', '중', '##기의', '무', '##신이', '##다', '.']
[708, 1254, 7590, 2000, 756, 2603, 455, 8452, 1012, 16]


In [35]:
class Tokenizer:
    def __init__(self):
        self.tokenizer_type_list = ["word", "morph", "syllable", "jaso", "wordPiece"]
        self.pad_token = "<pad>"
        self.max_seq_length = 10
        self.padding = False
    def tokenize(self, text, tokenizer_type): 
        assert tokenizer_type in self.tokenizer_type_list, "정의되지 않은 tokenizer_type입니다."
        if tokenizer_type == "word":
            tokenized_text = text.split(" ")
        elif tokenizer_type == "morph":
            tokenized_text = [lemma[0] for lemma in mecab.pos(text)]
        elif tokenizer_type == "syllable":
            tokenized_text = list(text)
        elif tokenizer_type == "jaso":
            tokenized_text = list(hgtk.text.decompose(text))
        elif tokenizer_type == "wordPiece":
            tokenized_text = wp_tokenizer.encode(text).tokens
        if self.padding:
            tokenized_text += [self.pad_token] * (self.max_seq_length - len(tokenized_text))
            return tokenized_text[:self.max_seq_length]
        else:
            return tokenized_text[:self.max_seq_length]
    def batch_tokenize(self, texts, tokenizer_type):
        for i, text in enumerate(texts):
            texts[i] = self.tokenize(text, tokenizer_type)
        return texts

In [36]:
my_tokenizer = Tokenizer()
my_tokenizer.pad_token = "[PAD]"
my_tokenizer.max_seq_length = 10
my_tokenizer.padding = True

In [37]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "wordPiece"))
print(my_tokenizer.batch_tokenize(["이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다."], "wordPiece"))

['이', '##순', '##신은', '조선', '중', '##기의', '무', '##신이', '##다', '.']
[['이', '##순', '##신은', '조선', '중', '##기의', '무', '##신이', '##다', '.'], ['그는', '임진', '##왜', '##란을', '승리', '##로', '이끌었다', '.', '[PAD]', '[PAD]']]


구현된 tokenizing 함수들을 모두 확인해보겠습니다.

In [38]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "word"))
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "morph"))
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "syllable"))
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "jaso"))
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "wordPiece"))

['이순신은', '조선', '중기의', '무신이다.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
['이순신', '은', '조선', '중기', '의', '무신', '이', '다', '.', '[PAD]']
['이', '순', '신', '은', ' ', '조', '선', ' ', '중', '기']
['ㅇ', 'ㅣ', 'ᴥ', 'ㅅ', 'ㅜ', 'ㄴ', 'ᴥ', 'ㅅ', 'ㅣ', 'ㄴ']
['이', '##순', '##신은', '조선', '중', '##기의', '무', '##신이', '##다', '.']
